### In this notebook we show how to build a conversational RAG system using OCI GenAI service, OCI OpenSearch service and LangChain framework. 

# Import packages & Libs

In [35]:
import os
from typing import Any, Iterator, List
from tqdm import tqdm

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_community.embeddings import OCIGenAIEmbeddings
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_community.docstore.document import Document
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI

from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.oci_generative_ai import OCIGenAI
from langchain.chains import ConversationalRetrievalChain

# Setup

In [36]:
# Put your compartment id
compartment_id = "Your compartment_id"
# service endpoint
endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"
# model_id for embedding 
model_id_embedding ="cohere.embed-english-v3.0"
# model_id for generation
model_id_generation = "cohere.command-r-plus" 

# Put your OCI opensearch_url
opensearch_url="Your opensearch_url"

# Setup OpenSearch Username & Password and put them below.  
OPENSEAECH_USERNAME="Your OPENSEAECH_USERNAME="
OPENSEAECH_PASSWORD=" your OPENSEAECH_PASSWORD"

auth = (OPENSEAECH_USERNAME, OPENSEAECH_PASSWORD)

# OCI Embedding and Chat LLM

In [52]:
# OCI GenAI Embedding 
embeddings = OCIGenAIEmbeddings(
    model_id=model_id_embedding,
    service_endpoint=endpoint,
    compartment_id=compartment_id,
    model_kwargs={"input_type": "SEARCH_DOCUMENT"}
)

# OCI GenAI Chat LLM 
llm_model = ChatOCIGenAI(
            model_id=model_id_generation, 
            service_endpoint=endpoint,
            compartment_id=compartment_id,
            model_kwargs={"temperature": 0, "max_tokens": 500, 'top_p': 1.0},
            is_stream=False)

# OCI OpenSearch as Vector Store

In [40]:
# Connect to OpenSearch
db = OpenSearchVectorSearch(opensearch_url=opensearch_url, 
                            index_name="", 
                            embedding_function=embeddings, 
                            http_auth=auth)

# Data processing: loading & chunking  

In [41]:
# Loading PDF locally from pdf_url directory
def load_pdf_from_directory(pdf_url):
    pdf_docs = []
    print("Loading PDFs...")
    for file in tqdm(os.listdir(pdf_url)):
        if file.endswith('.pdf'):
            pdf_file_path = pdf_url + file
            print("document name:", str(file))
            try:
                loader = PyPDFLoader(pdf_file_path)
                pdf_docs.extend(loader.load_and_split())
            except Exception as e:
                print(e)
    return pdf_docs

In [42]:
# method to split and chunk
def doc_splitter(docs: List[Document], chunk_size: int, chunk_overlap: int) -> List[Document]:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    print("chunk_size:",chunk_size,", chunk_overlap:",chunk_overlap)
    texts = text_splitter.split_documents(docs)
    return texts

In [43]:
## method to store vectors in db
def store_vectors(pages, my_index_name):
    total_chunks = 0
    if len(pages) > 95:
        page_chunks = round(len(pages)/95)
    else:
         page_chunks = 1

    # logic to handle OCIGenAIEmbeddings 95 docs to embed at a time,  a limit enforced by OCI GenAI
    for page_chunk in range(page_chunks):
        selected_pages = pages[page_chunk*95:(page_chunk+1)*95]
        total_chunks+=len(selected_pages)
        docs = [doc.page_content for doc in selected_pages]
        
        db.add_texts(texts=docs, 
                     embedding=embeddings, 
                     opensearch_url=opensearch_url, 
                     index_name=my_index_name, 
                     http_auth=auth)

    db.client.indices.refresh(index=my_index_name)
    
    print('my index name:', my_index_name)
    print('Vector DB created and stored Successfully.')
    print("Total Number of chunks created:", total_chunks)

# Store the chunks into DB

In [44]:
# Main function
def main_split_store(pdf_dir_path: str, chunk_size: int, chunk_overlap: int, my_index_name:str):


    pdf_docs = load_pdf_from_directory(pdf_dir_path)
    print(f"Number of pdf pages are {len(pdf_docs)}")

    # parse and chunk pdf pages
    pdf_splitted_chunks = doc_splitter(pdf_docs, chunk_size, chunk_overlap)

    # Creating and storing vector DB
    store_vectors(pdf_splitted_chunks, my_index_name)

## Run with chunk size and chunk overlap

In [45]:
pdf_dir_path = "./data/"
chunk_size = 1000
chunk_overlap = 200
my_index_name = "index_1"
main_split_store(pdf_dir_path, chunk_size, chunk_overlap, my_index_name)

Loading PDFs...


  0%|          | 0/2 [00:00<?, ?it/s]

document name: Developing AI applications with OCI Generative AI and LangChain.pdf


100%|██████████| 2/2 [00:00<00:00,  2.27it/s]


Number of pdf pages are 81
chunk_size: 1000 , chunk_overlap: 200
my index name: index_1
Vector DB created and stored Successfully.
Total Number of chunks created: 240


# Add memory and retriever 

In [46]:
# Query Document
db = OpenSearchVectorSearch(opensearch_url=opensearch_url, 
                            index_name=my_index_name, 
                            embedding_function=embeddings, 
                            http_auth=auth)
# adding memory
memory = ConversationBufferWindowMemory(memory_key="chat_history", 
                                        k=3, 
                                        return_messages=True, 
                                        output_key='answer')

retriever = db.as_retriever(search_kwargs={'k': 3}, 
                            search_type='similarity', 
                            chain_type="map-rerank")

# Prompt for generation 

In [47]:
def get_prompt(num_words: int):
    """
    This funtion creates prompt template for cohere and attaches placeholders for the values to be updated later

    Returns:
        string: Prompt template
    """

    SYSTEM_PROMPT = """You are a chatbot. Your task is to help answer queries using the below given context. 
    If there is anything that you cannot answer, or you think is inappropriate to answer, simply reply as, 
    "Sorry, I cannot help you with that."""
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

    SYSTEM_PROMPT_template = B_SYS + SYSTEM_PROMPT + E_SYS


    context_instruction_template = "CHAT HISTORY: {chat_history}\n----------\nCONTEXT: {context}\n----------\n\nInstructions:1. Answer only from the given context.\n             2: Do not generate any new content out of this context.\n             3: Your answer should not include any harmful, unethical, violent, racist, sexist, pornographic, toxic, discriminatory, blasphemous, dangerous, or illegal content.\n             4: Please ensure that your responses are socially unbiased and positive in nature.\n             5: Ensure length of the answer is within " + str(num_words) +" words.\n\nNow, Answer the following question: {question}\n"
    prompt_template =  '<s>'+B_INST + SYSTEM_PROMPT_template + context_instruction_template + E_INST

    return prompt_template

In [49]:
prompt_template = get_prompt(300)
print(prompt_template)
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "chat_history","num_words", "question"])

<s>[INST]<<SYS>>
You are a chatbot. Your task is to help answer queries using the below given context. 
    If there is anything that you cannot answer, or you think is inappropriate to answer, simply reply as, 
    "Sorry, I cannot help you with that.
<</SYS>>

CHAT HISTORY: {chat_history}
----------
CONTEXT: {context}
----------

Instructions:1. Answer only from the given context.
             2: Do not generate any new content out of this context.
             3: Your answer should not include any harmful, unethical, violent, racist, sexist, pornographic, toxic, discriminatory, blasphemous, dangerous, or illegal content.
             4: Please ensure that your responses are socially unbiased and positive in nature.
             5: Ensure length of the answer is within 300 words.

Now, Answer the following question: {question}
[/INST]


# Conversational Retrieval Chain 

In [50]:
qa_chain = ConversationalRetrievalChain.from_llm(llm=llm_model,
                                                    retriever=retriever,
                                                    memory=memory,
                                                    combine_docs_chain_kwargs={"prompt": prompt},
                                                    return_source_documents=True,
                                                    verbose=True)
response = qa_chain.invoke({"question": "What is Oracle Cloud?"})
source_documents = response.get("source_documents")
print(response.get("answer"))




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST]<<SYS>>
You are a chatbot. Your task is to help answer queries using the below given context. 
    If there is anything that you cannot answer, or you think is inappropriate to answer, simply reply as, 
    "Sorry, I cannot help you with that.
<</SYS>>

CHAT HISTORY: 
----------
CONTEXT: 1
Welcome to Oracle Cloud
Oracle Cloud is the industry's broadest and most integrated cloud provider, with deployment
options ranging from the public cloud to your data center. Oracle Cloud offers best-in-class
services across Software as a Service (SaaS), Platform as a Service (PaaS), and
Infrastructure as a Service (IaaS).
Topics
•About Oracle Cloud
•Overview of Oracle Cloud Subscriptions
•About Oracle Cloud Accounts
•Oracle Cloud Terminology
•How Do I Sign Up?
•Contact Oracle Support
About Oracle Cloud
Oracle Cloud is one of the few cloud providers that can offer a complete set of cloud

In [33]:
print(source_documents)

[Document(page_content="1\nWelcome to Oracle Cloud\nOracle Cloud is the industry's broadest and most integrated cloud provider, with deployment\noptions ranging from the public cloud to your data center. Oracle Cloud offers best-in-class\nservices across Software as a Service (SaaS), Platform as a Service (PaaS), and\nInfrastructure as a Service (IaaS).\nTopics\n•About Oracle Cloud\n•Overview of Oracle Cloud Subscriptions\n•About Oracle Cloud Accounts\n•Oracle Cloud Terminology\n•How Do I Sign Up?\n•Contact Oracle Support\nAbout Oracle Cloud\nOracle Cloud is one of the few cloud providers that can offer a complete set of cloud services\nto meet all your enterprise computing needs.\nUse Oracle Infrastructure as a Service (IaaS) offerings to quickly set up the virtual machines,\nstorage, and networking capabilities you need to run just about any kind of workload. Your\ninfrastructure is managed, hosted, and supported by Oracle.\nUse Oracle Platform as a Service offerings to provision rea

In [51]:
print(response.get("answer"))

Oracle Cloud is a cloud computing service offered by Oracle Corporation, which provides a wide range of cloud services to meet various enterprise computing needs. It is one of the few cloud providers that offer a complete set of cloud services, including Software as a Service (SaaS), Platform as a Service (PaaS), and Infrastructure as a Service (IaaS). 

With Oracle Cloud, you can quickly set up virtual machines, storage, and networking capabilities to run almost any kind of workload. Your infrastructure is managed, hosted, and supported by Oracle, ensuring that your enterprise computing needs are met efficiently and effectively. 

Oracle Cloud also offers ready-to-use environments and tools for developers, making it easier to build and deploy applications. It provides deployment options ranging from the public cloud to data centers, giving you flexibility in choosing the right option for your business.


- ## Now is your turn to do the following experimentations using the above codes: 
    - ### Change `chunk_size`, `chunk_overlap` in `main_split_store` and re-run the experiment 
    - ### Change the value of `search_kwargs` (k) in `retriever` and re-run the experiment 